In [5]:
import os.path
import numpy as np
import pandas as pd

from source.Files_operating import read_dataFile

In [20]:
interval_path = "data/intervals/"
signal_path = "data/d-alpha/"

filename = "sht44194"
F_ID = filename[-5:]

if os.path.isfile(f"data/intervals/df/{F_ID}_data.csv"):
    df = pd.read_csv(f"data/intervals/df/{F_ID}_data.csv", sep=",")
else:
    df = read_dataFile(interval_path + filename + "_exportGlobus2.dat", F_ID)
    df["ch1_marked"] = pd.Series([0] * df.shape[0])
    df["ch1_ai_marked"] = pd.Series([0] * df.shape[0])

df.head()

,t,ch1,ch1_marked,ch1_ai_marked
0,164.646,0.33486,0,0
1,164.647,0.29907,0,0
2,164.648,0.31952,0,0
3,164.649,0.32208,0,0
4,164.650,0.35275,0,0


In [26]:
def normalise_series(data: np.array) -> np.array:
    max_point, min_point, median_value = data.max(), data.min(), np.median(data)
    return (data - median_value) / abs(max_point - min_point)

def get_boards(data: np.array, scale=np.exp(1)):
    loc_max_ind = np.argmax(data)
    dist_ind = np.argsort(np.abs(data - data[loc_max_ind] / scale))
    return [dist_ind[dist_ind < loc_max_ind][0], dist_ind[dist_ind > loc_max_ind][0]]

In [31]:
df["ch1_marked"] = pd.Series([0] * df.shape[0])

In [ ]:
## import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

l_edge = int(input(f"Input start index [0:{df.shape[0]//1000*1000}]: ").strip().split()[0])
step = 1500
width = 2000

step_away = 10

while l_edge < df.shape[0]:
    fig, ax = plt.subplots()

    fig.set_figwidth(16)
    fig.set_figheight(6)

    r_edge = min(l_edge + width, df.shape[0])

    ax.plot(range(l_edge, r_edge), normalise_series(df.ch1)[l_edge:r_edge], label="D-alpha")
    # ax.plot(range(l_edge, r_edge), df.ch1_filter_norm[l_edge:r_edge], label="Filtered D-alpha")
    ax.plot(range(l_edge, r_edge), df.ch1_marked[l_edge:r_edge] * 0.5, label="Marked D-alpha")

    ax.grid(which='major', color='#DDDDDD', linewidth=0.9)
    ax.grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.7)
    ax.minorticks_on()
    ax.xaxis.set_minor_locator(AutoMinorLocator(10))
    ax.legend()
    plt.show()

    res = list(map(int, input("Input index pairs of ELM fragments:\n").strip().split()))
    for i in range(0, len(res), 2):
        board_ind = get_boards(df.loc[res[i]:res[i + 1], "ch1"].to_numpy(), scale=1.5)
        board_ind[0] = max(board_ind[0] - step_away + res[i], 0)
        board_ind[1] += res[i]
        
        # print(board_ind)
        
        if (df.loc[board_ind[0]:board_ind[1], "ch1_marked"] == 1.).all():
            df.loc[board_ind[0]:board_ind[1], "ch1_marked"] = 0.
        elif (df.loc[board_ind[0]:board_ind[1], "ch1_marked"] == 1.).any():
            if input("Input 1 or 0 to set: ").lower() in ["1", "1.0", "y", "н", "1."]:
                df.loc[board_ind[0]:board_ind[1], "ch1_marked"] = 1.
            else:
                df.loc[board_ind[0]:board_ind[1], "ch1_marked"] = 0.
        else:
            df.loc[board_ind[0]:board_ind[1], "ch1_marked"] = 1.

    plt.close()
    plt.clf()
    l_edge += step

# 275 875 925 1425 1425 1800 1800 2000

In [36]:
df.to_csv(f"data/intervals/df/{F_ID}_data.csv", index=False)
df.describe()

,t,ch1,ch1_marked,ch1_ai_marked
count,76155.000000,76155.000000,76155.000000,76155.0
mean,202.723000,0.465146,0.180211,0.0
std,21.984199,0.183108,0.384366,0.0
min,164.646000,0.181490,0.000000,0.0
25%,183.684500,0.334860,0.000000,0.0
50%,202.723000,0.393650,0.000000,0.0
75%,221.761500,0.570020,0.000000,0.0
max,240.800000,1.283200,1.000000,0.0


In [12]:
count = 0
for i in range(df.shape[0]-1):
    if df.iloc[i].ch1_marked == 0 and df.iloc[i+1].ch1_marked == 1:
        count += 1
print(count)

68
